# Notebook: Train Model

## Packages

In [1]:
from load_dataset_folds import load_dataset_folds
from ACD import train_ACD_model
from OTE import train_OTE_model
import pandas as pd
import numpy as np
import constants
import warnings
import shutil
import torch
import json
import sys

* Todo: "output2" umbenennen ✅
* Todo: Evaluationsskript anpassen
* Todo: "OTD" entfernen ✅
* Todo: Metrics speichern
* Todo: Pfade ändern real/fake data
* Todo: Nochmal prüfen, ob korrekte daten geladen werden
* Todo: Schauen, wieso der erste Term mehrere Tags erhält
* Todo: F1 macro brechnen
* Metriken bei acd checken und schauen, ob evtl standardmethoden verwendet werden können

## Parameters

In [2]:
LLM_NAME = "Llama13B"
N_REAL = 500
N_SYNTH = 0
TARGET = "aspect_term" # "aspect_term", "aspect_category"
LLM_SAMPLING = "fixed"

## Settings

In [3]:
# Set seeds
torch.manual_seed(constants.RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(constants.RANDOM_SEED)

# Ignore warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="transformers.optimization")
torch.device("mps")

# Disable Pycache
sys.dont_write_bytecode = True

## Code

In [4]:
train_dataset, test_dataset = load_dataset_folds(LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING)

### Load Model

In [5]:
if TARGET == "aspect_category":
   results = train_ACD_model(LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)

In [6]:
if TARGET == "aspect_term":
    results = train_OTE_model(LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)

Some weights of the model checkpoint at deepset/gbert-base were not used when initializing BertForSpanCategorizationOTE: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSpanCategorizationOTE from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSpanCategorizationOTE from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSpanCategorizationOTE were not initialized from the model checkpoint at de

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/32 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 473.264, 'learning_rate': 0.0, 'epoch': 1.0}


...

{'loss': 473.265, 'learning_rate': 0.0, 'epoch': 1.0}


  0%|          | 0/32 [00:00<?, ?it/s]

GENERAL-IMPRESSION dataset size: 37
FOOD dataset size: 167
SERVICE dataset size: 169
AMBIENCE dataset size: 99
PRICE dataset size: 33
{'eval_loss': 473.1297607421875, 'eval_f1_GENERAL-IMPRESSION': 0.13333333333333333, 'eval_precision_GENERAL-IMPRESSION': 0.07142857142857142, 'eval_recall_GENERAL-IMPRESSION': 1.0, 'eval_tp_GENERAL-IMPRESSION': 37, 'eval_tn_GENERAL-IMPRESSION': 0, 'eval_fp_GENERAL-IMPRESSION': 481, 'eval_fn_GENERAL-IMPRESSION': 0, 'eval_n_samples_GENERAL-IMPRESSION': 37, 'eval_f1_FOOD': 0.015220700152207002, 'eval_precision_FOOD': 0.007947019867549669, 'eval_recall_FOOD': 0.17964071856287425, 'eval_tp_FOOD': 30, 'eval_tn_FOOD': 0, 'eval_fp_FOOD': 3745, 'eval_fn_FOOD': 137, 'eval_n_samples_FOOD': 167, 'eval_f1_SERVICE': 0, 'eval_precision_SERVICE': 0.0, 'eval_recall_SERVICE': 0.0, 'eval_tp_SERVICE': 0, 'eval_tn_SERVICE': 0, 'eval_fp_SERVICE': 6077, 'eval_fn_SERVICE': 169, 'eval_n_samples_SERVICE': 169, 'eval_f1_AMBIENCE': 0.019607843137254898, 'eval_precision_AMBIENCE': 0

  0%|          | 0/32 [00:00<?, ?it/s]

GENERAL-IMPRESSION dataset size: 37
FOOD dataset size: 167
SERVICE dataset size: 169
AMBIENCE dataset size: 99
PRICE dataset size: 33


### Save Results

In [7]:
with open(f'results_json/results_{LLM_NAME}_real{N_REAL}_synth{N_SYNTH}_{TARGET}_{LLM_SAMPLING}.json', 'w') as json_file:
    json.dump(results, json_file)

df = pd.DataFrame([results])
df.to_csv(f'results_csv/results_{LLM_NAME}_real{N_REAL}_synth{N_SYNTH}_{TARGET}_{LLM_SAMPLING}.csv', index=False)

In [8]:
results

{'LLM_NAME': 'Llama13B',
 'N_REAL': 500,
 'N_SYNTH': 0,
 'TARGET': 'aspect_term',
 'LLM_SAMPLING': 'fixed',
 'eval_f1_micro': 0.04060766177585412,
 'eval_f1_macro': 0.032486129420683296,
 'eval_precision': 0.006251538828448222,
 'eval_precision_GENERAL-IMPRESSION': 0.07142857142857142,
 'eval_precision_FOOD': 0.003973509933774834,
 'eval_precision_SERVICE': 0.0020327957717847945,
 'eval_precision_AMBIENCE': 0.009080820657175337,
 'eval_precision_PRICE': 0.0,
 'eval_recall_GENERAL-IMPRESSION': 1.0,
 'eval_recall_FOOD': 0.08982035928143713,
 'eval_recall_SERVICE': 0.08875739644970414,
 'eval_recall_AMBIENCE': 0.24813432835820895,
 'eval_recall_PRICE': 0.0,
 'eval_f1_GENERAL-IMPRESSION': 0.13333333333333333,
 'eval_f1_FOOD': 0.007610350076103501,
 'eval_f1_SERVICE': 0.00397456279809221,
 'eval_f1_AMBIENCE': 0.017512400895887435,
 'eval_f1_PRICE': 0.0,
 'runtime': 121.91285920143127,
 'runtime_formatted': '2m 1s',
 'eval_loss': 473.12806701660156}

### Remove useless folders

In [9]:
try:
    shutil.rmtree("outputs")
except:
    pass